# Iterators and Generators
Yang Xi <br>
14 Sep, 2020


<br>


* Introduction
* Iterators
	* Introduction
* Generators
	* Definition
	* Example: Fibonacci Sequence
	* `send` Method / Coroutines
	* `throw` Method
	* `yield from`

# Introduction
A **iterator** is an object that can be iterated over like we do in a for loop.<br>
Iterators work on *lazy evaluation* principle, which delays the evaluation of an expression until its value is really needed. This is a useful as it reduces the memory consumption of the object.<br>
An iterator is an abstraction, which enables the programmer to access all the elements of an iterable object (a set, a string, a list etc.) without any deeper knowledge of the data structure of this object.

**Generators** are a special kind of function,which enable us to implement or generate iterators.

Reference: https://www.python-course.eu/python3_generators.php

# Iterators
Iterators are mostly **implicitly** used, like in for-loop. In the example below, we are iterating over alist, but take note: a list is **not** an iterator, while it can be used like an iterator.

In [1]:
cities = ["Paris", "Berlin", "Hamburg"]
for location in cities:
    print("location: " + location)

location: Paris
location: Berlin
location: Hamburg


What happened when the for loop is executed?
1. Apply function `iter` to the object `o` following the `in` keyword.
    * If `o` is not iterable, raise an exception.
2. Call `iter(o)` to return an iterator `iterator_obj`.
3. Iterator over object `o` using the `next` method.
4. Stop the loop when `next(iterator_obj)` is exhausted - `StopIteration` exception.

Code below demonstrates this behavior:

In [2]:
cities = ["Paris", "Berlin", "Hamburg"]
cities_iterator = iter(cities)
next(cities_iterator)

'Paris'

In [3]:
next(cities_iterator)

'Berlin'

In [4]:
next(cities_iterator)

'Hamburg'

In [5]:
next(cities_iterator)

StopIteration: 

Dictionary data type `dict` supports iterators as well:

In [6]:
capitals = {"France":"Paris", "Netherlands":"Amsterdam", "Germany":"Berlin"}
for country in capitals:
     print("The capital city of " + country + " is " + capitals[country])

The capital city of France is Paris
The capital city of Netherlands is Amsterdam
The capital city of Germany is Berlin


# Generators
### Definition
**Generator** is the usual and easiest way to create an iterator in Python.

A generator is a function which returns a generator object. This **generator object** is like a function which produces a sequence of values, instead of a single object.<br>
This sequence of values is produced by iterating over it, e.g. with a for loop. And the value, on which can be iterated, are created by the `yield` statement.

Upon reaching the `yield` statement, the value behind the `yield` statement is returned, and the execution of the generator is interrupted.<br>
When `next` is called on the generator object, the generator function will resume right after the `yield` statement where last call was made, and **all local variables still exist**.

If there is a `return` statement in the generator, the execution will stop with a `StopIteration` exception.

Iterators can also be implemented with a class. However, the crucial advantage of generators consists in automatically creating the methods `__iter__()` and `__next__()` in a very neat manner.

In [7]:
def city_generator():
    yield("Hamburg")
    yield("Konstanz")

city = city_generator()

In [8]:
next(city)

'Hamburg'

In [9]:
next(city)

'Konstanz'

In [10]:
next(city)

StopIteration: 

There is no "reset" of a generator, but this can be done by create another generator, e.g. `city = city_generator()` again.

### Example: Fibonacci Sequence

In [11]:
def fibonacci(n):
    a, b, counter = 0, 1, 0
    while True:
        if (counter > n): 
            return
        yield a
        a, b = b, a + b
        counter += 1

f = fibonacci(5)
for x in f:
    print(x, end=" ")

0 1 1 2 3 5 

### `send` Method / Coroutines
The `send()` method sends a message (object) to the generateor, and returns the value yielded by the generator:

In [12]:
def simple_coroutine():
    print("coroutine has been started!")
    while True:
        x = yield "foo"
        print("coroutine received: ", x)

cr = simple_coroutine()

In [13]:
next(cr)

coroutine has been started!


'foo'

In [14]:
ret_value = cr.send("Hi")
ret_value

coroutine received:  Hi


'foo'

Note that we had to call `next` on the generator first to start it. Using `send` to a generator not started yet will lead to an exception.

The `next` call always sends a `None` object.

We can use `send` to modify the variables saved in the generator:

In [15]:
def count(first_val=0):
    counter = first_val
    while True:
        new_counter_val = yield counter
        if new_counter_val is None:
            counter += 1
        else:
            counter = new_counter_val

counter = count(0)
for i in range(2):
    print(next(counter))

0
1


In [16]:
counter.send(5)

5

In [17]:
next(counter)

6

### `throw` Method
The `throw()` method raised an excemption at the point where the generator was paused, and returns the next value yielded by the generator.

We can use `throw` to exam variables inside the generator:

In [18]:
def count(first_val=0):
    counter = first_val
    while True:
        try:
            new_counter_val = yield counter
            if new_counter_val is None:
                counter += 1
            else:
                counter = new_counter_val
        except Exception:
            yield (first_val, counter)

counter = count(0)
for i in range(2):
    print(next(counter))

0
1


In [19]:
status = counter.throw(Exception)
status

(0, 1)

In [20]:
next(counter)

1

### `yield from`
`yield from <expr>` provides a shortcut to yield from an iterable. The `<expr>` has to be an expression evaluating to an iterable.

In [21]:
def gen():
    yield from [1,2,3]

g = gen()
for x in g:
    print(x, end=" ")

1 2 3 

If the `<expr>` is another generator (subgenerator), it is allowed to execute a `return` statement:

In [22]:
def subgenerator():
    yield 1
    return 42

def gen():
    x = yield from subgenerator()
    print(x)

for x in gen():
    print(x)

1
42
